In [1]:
import pandas as pd
import numpy as np

## Read in data

Dummy dataset included for testing. 

This has been written on the assumption that you are working with page views & click events, but you can run this only using pageview data. 

In [2]:
customEvents = pd.read_csv('data_test/eventsDemo.csv')
pageViews = pd.read_csv('data_test/pageViewsDemo.csv')

customEvents.timestamp = customEvents.timestamp.apply(lambda x : pd.to_datetime(x))
pageViews.timestamp = pageViews.timestamp.apply(lambda x : pd.to_datetime(x))

In [3]:
customEvents.head(2)

,timestamp,name,session_Id,user_Id
0,2024-07-07 13:03:42.622000+00:00,Calculate dates,w9MzPUanCt/nkBkXLtpxq+,hqFhE9wWs6cVoHMXR8ev6c
1,2024-07-07 13:03:44.566000+00:00,Reason 13,w9MzPUanCt/nkBkXLtpxq+,hqFhE9wWs6cVoHMXR8ev6c


In [4]:
pageViews.head(2)

,timestamp,name,session_Id,user_Id
0,2024-07-06 06:58:12.505000+00:00,Check information,xBt2AIXOtvlCMfLZgDSKNO,S9y1hcrJytHsZYcLjKSsxc
1,2024-07-06 06:59:09.216000+00:00,Get calculation reason,Qa/eiaFhg83kolPMjB9/FG,hKrzluLLYRQuUplqVDFdjK


In [5]:
# functions below expect a type column 
# even if you are only running this view pageview data, include a type 

customEvents['type'] = 'customEvent' 
pageViews['type'] = 'pageView' 

In [6]:
# convert custom events to more readable names: this is project specific, e.g:

custom_event_dict = {
    'manual-entry': 'Manual entry', 
    'tab_detailed': 'Detailed tab'
}

customEvents['name'] = customEvents['name'].apply(lambda x: custom_event_dict.get(x, x))

# In case there are equivalent page views and click events 
customEvents['name'] = customEvents['name'].apply(lambda x: 'Click: ' + x)

In [7]:
# convert page views to more readable names: this is project specific - e.g: 

page_view_dict = {
    'Review calculation': 'Review information'
}
pageViews['name'] = pageViews['name'].apply(lambda x: page_view_dict.get(x, x))

## Remove bot / testing users

In [8]:
# This step is option and should be adapted for your project purposes. 
# In this example case, we do not expect to see any ordinary users with more than 50 pageviews per session 

max_views = 50

relevant_sessions = pageViews.groupby('session_Id').session_Id.count()[pageViews.groupby('session_Id').session_Id.count() < max_views].index.to_list()

pageViews = pageViews[pageViews.session_Id.isin(relevant_sessions)].copy()
customEvents = customEvents[customEvents.session_Id.isin(relevant_sessions)].copy()

assert len(pageViews.groupby('session_Id').session_Id.count()[pageViews.groupby('session_Id').session_Id.count() > max_views].sort_values()) == 0

## Combine page views and custom events  

In [9]:
# # combine without the duration column
# df = pd.concat([pageViews[[c for c in pageViews if c != 'duration']], customEvents])

# # convert timestamp from string to datetime
# df.timestamp = df.timestamp.apply(lambda x : pd.to_datetime(x))

## Or Choose Specific CustomEvents to Include 

In [10]:
relevant_events = ['Click: Service Home', 'Click: Print slip']
relevant_customEvents = customEvents[customEvents.name.isin(relevant_events)]


# determine size of dataframe prior to and after dropping extra events
print(customEvents.shape[0], relevant_customEvents.shape[0])

# combine 
df = pd.concat([pageViews, relevant_customEvents])

16870 1974


## Calculate page view average duration

In [11]:
# reworked to respect click events 
def return_page_visit_duration(df, min_time = 1, max_time = 3600): 
    """
    Function returns a dataframe with mean and median page visit time for each 
    page visited, differentiated by which step of the journey it is. 
    """    
    
    df_all = df.copy()
    df_all.sort_values(by = ['session_Id', 'timestamp'], inplace=True)
    
   #  add the step and label to each one 
    df_all['step'] = df_all.groupby('session_Id').cumcount() + 1
    df_all['step'] = df_all['step'].apply(lambda x: str(x))
    

    # now that we have correct labels, drop everything apart from pageViews - this stops clickEvents from skewing the view time 
    df_all['label'] = df_all.apply(lambda x : '{}: '.format(x.step) + x['name'], axis = 1)

    df_views = df_all[df_all.type == 'pageView'].copy()
    
    # convert timestamp from string to datetime and find the next timestamp
    df_views.timestamp = df_views.timestamp.apply(lambda x : pd.to_datetime(x))
    df_views['next_timestamp'] = df_views.groupby('session_Id').timestamp.shift(-1)
    
    # and calculate duration: 
    df_views['visitTimeSeconds'] = (df_views.next_timestamp - df_views.timestamp).apply(lambda x : x.total_seconds())

    # remove pageViews that are < min_time or > max_time

    print("Original dataframe size: {} rows".format(df_views.shape[0]))
    reduced = df_views[(df_views.visitTimeSeconds >= min_time) & (df_views.visitTimeSeconds <= max_time)]
    print("Reduced size: {} rows".format(reduced.shape[0]))
    print("Removed {} pageViews with a duration less than {} second or greater than {} seconds".format(df_views.shape[0] - reduced.shape[0], min_time, max_time))
    
    # average duration by labels..include count to remove outliers 
    pageVisitTime = reduced.groupby('label').visitTimeSeconds.agg(['mean', 'median', 'count'])
    pageVisitTime = pageVisitTime[pageVisitTime['count'] >= 2]
    
    return pageVisitTime

In [12]:
# If you want to test the output of this function, you can uncomment the code below and try out different parameters 

# pageVisitTime = return_page_visit_duration(df, min_time = 0)
# pageVisitTime = return_page_visit_duration(df, min_time = 1)

## Reformat data for sankey 

In [13]:
# have cancelled out values greater than 1 hour so don't need to consider how to format hours here  

def format_time(x): 
    """
    Function returns seconds formatted as seconds, or minutes and seconds.
    """    
    if x < 60: 
        return f'{int(round(x, 0))} secs'
    if x >= 60: 
        mins = int(x // 60)
        secs = int(round(x % 60, 0))
        return f'{mins}.{secs} mins'

In [14]:
def filter_starting_step(x, starting_step, n_steps):
    """
    Function used to return the first n_steps for each user starting from the "starting_step".
    The function will be used to generate the event sequence journey for each user.
    """
    starting_step_index = x.index(starting_step)
    
    return x[starting_step_index: starting_step_index + n_steps] 

In [15]:
def user_journey(events, starting_step, n_steps=3, events_per_step=5):
    """
    Function used to map out the journey for each user starting from the defined "starting_step" and count
    how many identical journeys exist across users.
    """
    # sort events by time
    events = events.sort_values(['session_Id', 'timestamp'])
    # find the users that have performed the starting_step
    valid_ids = events[events['name'] == starting_step]['session_Id'].unique()

    # plan out the journey per user, with each step in a separate column
    flow = events[(events['session_Id'].isin(valid_ids))] \
        .groupby('session_Id') \
        .name.agg(list) \
        .to_frame()['name'] \
        .apply(lambda x: filter_starting_step(x, starting_step=starting_step, n_steps=n_steps)) \
        .to_frame() \
        ['name'].apply(pd.Series)

    # fill NaNs with "End" to denote no further step by user; this will be filtered out later
    flow = flow.fillna('End')

    # add the step number as prefix to each step
    for i, col in enumerate(flow.columns):
        flow[col] = '{}: '.format(i + 1) + flow[col].astype(str)

    # replace events not in the top "events_per_step" most frequent list with the name "Other"
    # this is done to avoid having too many nodes in the sankey diagram
    for col in flow.columns:
        all_events = flow[col].value_counts().index.tolist()
        all_events = [e for e in all_events if e != (str(col + 1) + ': End')]
        top_events = all_events[:events_per_step]
        to_replace = list(set(all_events) - set(top_events))
        flow[col].replace(to_replace, [str(col + 1) + ': Other'] * len(to_replace), inplace=True)

    # count the number of identical journeys up the max step defined
    flow = flow.groupby(list(range(n_steps))) \
        .size() \
        .to_frame() \
        .rename({0: 'count'}, axis=1) \
        .reset_index()

    return flow

In [16]:
def sankey_df(df, starting_step, n_steps=6, events_per_step=5, hide_ends = False):
    """
    Function used to generate the dataframe needed to be passed to the sankey generation function.
    "source" and "target" column pairs denote links that will be shown in the sankey diagram.
    """
    # generate the user user flow dataframe
    flow = user_journey(df, starting_step, n_steps, events_per_step)
    
    pageVisitTime = return_page_visit_duration(df, min_time = 0, max_time = 3600)

    # create a dictionary of node types to determine colour 
    node_type = df.groupby(['name', 'type'])['session_Id'].count().sort_values(ascending = False)
    node_type = node_type[node_type > 10] # drop any small values 
    node_type_dict = node_type.reset_index().set_index('name')['type'].to_dict()
    # add end 
    node_type_dict['End'] = 'End' 
    node_type_dict['Click'] = 'customEvent' # adding in separately as click events get 'Click: ' added to their label. 


    # create the basic nodes labels list
    label_list = []
    cat_cols = flow.columns[:-1].values.tolist()
    for cat_col in cat_cols:
        label_list_temp = list(set(flow[cat_col].values))
        label_list = label_list + label_list_temp

    # create a list of colours for the nodes
    type_list = [node_type_dict.get(label.split(': ')[1], 'Other') for label in label_list]
    colors_list = ['indigo' if i == 'pageView' else 'teal' if i == 'customEvent' else 'mediumvioletred' if i == 'End' else 'grey' for i in type_list]
    
    # add page duration onto the labels for pageView types
    label_list_with_time = []

    pageViewTemplate = "{}<br>Median view time: {}<br>Mean view time: {}"
    customEventTemplate = "{}"



    for l, t in zip(label_list, type_list): 
        if t == 'pageView': 
            medianT = round(pageVisitTime.to_dict()['median'][l], 1)
            meanT = round(pageVisitTime.to_dict()['mean'][l], 1)
            label_list_with_time.append(pageViewTemplate.format(l, format_time(medianT), format_time(meanT) ))  
        else: 
            label_list_with_time.append(customEventTemplate.format(l)) 

    # transform flow df into a source-target pair
    for i in range(len(cat_cols) - 1):
        if i == 0:
            source_target_df = flow[[cat_cols[i], cat_cols[i + 1], 'count']]
            source_target_df.columns = ['source', 'target', 'count']
        else:
            temp_df = flow[[cat_cols[i], cat_cols[i + 1], 'count']]
            temp_df.columns = ['source', 'target', 'count']
            source_target_df = pd.concat([source_target_df, temp_df])
        source_target_df = source_target_df.groupby(['source', 'target']).agg({'count': 'sum'}).reset_index()

    # add index for source-target pair
    source_target_df['source_id'] = source_target_df['source'].apply(lambda x: label_list.index(x))
    source_target_df['target_id'] = source_target_df['target'].apply(lambda x: label_list.index(x))
    
    # filter out the end step if requested 
    # bug: if you provide a node without a target, it forces the node to the end of the visualization. 
    # So you cannot simply filter out the circular / target end nodes..
    if hide_ends == True: 
        source_target_df = source_target_df[(~source_target_df['source'].str.contains('End')) &
                                            (~source_target_df['target'].str.contains('End'))]


    return label_list_with_time, colors_list, source_target_df

In [17]:
# If you want to test the output of this function, you can uncomment the code below and try out different parameters 

# label_list, colors_list, source_target_df = sankey_df(df, starting_step = 'Home', n_steps=6, events_per_step=8)

## Plotly visualization code 

In [18]:
def plot_user_flow(df, starting_step, n_steps=3, events_per_step=5, hide_ends = False, title='Sankey Diagram'):
    """
    Function used to generate the sankey plot for user journeys.
    """
    # transform raw events dataframe into  source:target pairs including node ids and count of each combination
    label_list, colors_list, source_target_df = sankey_df(df, starting_step, n_steps, events_per_step, hide_ends = False)

    # creating the sankey diagram
    data = dict(
        type='sankey',
        node=dict(
            pad=60,
            thickness=20,
            color=colors_list,
            line=dict(
                color="black",
                width=0.5
            ),
            label=label_list, 
            customdata = ['Page View<br>{}'.format(l) if 'view' in l else 'Click Event<br>{}'.format(l) for l in label_list], 
            hovertemplate='%{customdata}', 
        ),
        link=dict(
            source=source_target_df['source_id'].values.tolist(),
            target=source_target_df['target_id'].values.tolist(),
            value=source_target_df['count'].astype(int).values.tolist(),
            color = '#e6e6e6', # between gainsboro and whitesmoke. try also 'whitesmoke', 'gainsboro'
            hovercolor = 'dimgrey', 
            hoverlabel=dict(
                bgcolor='#C2C4C7'), 
            

        )
    )

    # set window width so that steps are evenly spaced out
    if n_steps < 5:
        width = None
    else:
        width = n_steps * 200

    layout = dict(
        height=1000,
        width=width,
        margin=dict(t=100, l=40, r=200, b=30),
        title=title,
        font=dict(
            size=13, 
            family='Arial, sans-serif', 
        ), 
        title_font = dict(
            size = 24, 
            family='Arial, sans-serif', 
            weight = 'bold', 
            color = 'indigo', 
        )
    )

    fig = dict(data=[data], layout=layout)
    return fig

## Install necessary visualization packages

In [ ]:
# uncomment the below to run in the notebook, alternatively run in a terminal without the % 
# %pip install chart-studio 

In [19]:
from chart_studio import plotly 

from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected='true')

## Plot data 

In [21]:
fig = plot_user_flow(df, starting_step = 'Home', n_steps=10, events_per_step=5, hide_ends = False, 
                     title='Test User Flow<br><sup>Page views (purple) and click events (teal) July 1st - 7th 2024</sup>') 

Original dataframe size: 13433 rows
Reduced size: 11825 rows
Removed 1608 pageViews with a duration less than 0 second or greater than 3600 seconds


In [22]:
# this saves the figure to an html file, which can then be opened in its own tab or in the browser 
plot(fig, filename='user_flow_outputs/demoUserFlow.html')

'user_flow_outputs/demoUserFlow.html'